In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.2.1 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
from pyspark.sql import functions as F

In [0]:
F.__dir__()

Out[11]: ['__name__',
 '__doc__',
 '__package__',
 '__loader__',
 '__spec__',
 '__file__',
 '__cached__',
 '__builtins__',
 'sys',
 'functools',
 'warnings',
 'since',
 'SparkContext',
 'PythonEvalType',
 'Column',
 '_to_java_column',
 '_to_seq',
 '_create_column_from_literal',
 'DataFrame',
 'StringType',
 'DataType',
 'UserDefinedFunction',
 '_create_udf',
 '_create_edge_udf',
 'pandas_udf',
 'PandasUDFType',
 'to_str',
 'DeprecatedFuncUsageLogger',
 '_get_get_jvm_function',
 '_invoke_function',
 '_invoke_function_over_column',
 '_invoke_binary_math_function',
 '_options_to_str',
 'lit',
 'col',
 'column',
 'asc',
 'desc',
 'sqrt',
 'abs',
 'max',
 'min',
 'count',
 'sum',
 'avg',
 'mean',
 'sumDistinct',
 'sum_distinct',
 'product',
 'acos',
 'acosh',
 'asin',
 'asinh',
 'atan',
 'atanh',
 'cbrt',
 'ceil',
 'cos',
 'cosh',
 'cot',
 'csc',
 'exp',
 'expm1',
 'floor',
 'log',
 'log10',
 'log1p',
 'rint',
 'sec',
 'signum',
 'sin',
 'sinh',
 'tan',
 'tanh',
 'toDegrees',
 'toRadians',


In [0]:
cars = spark.sql('select * from default.cars')
cars.printSchema()

root
 |-- dateCrawled: string (nullable = true)
 |-- name: string (nullable = true)
 |-- seller: string (nullable = true)
 |-- offerType: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- abtest: string (nullable = true)
 |-- vehicleType: string (nullable = true)
 |-- yearOfRegistration: integer (nullable = true)
 |-- gearbox: string (nullable = true)
 |-- powerPS: integer (nullable = true)
 |-- model: string (nullable = true)
 |-- kilometer: integer (nullable = true)
 |-- monthOfRegistration: integer (nullable = true)
 |-- fuelType: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- notRepairedDamage: string (nullable = true)
 |-- dateCreated: string (nullable = true)
 |-- postalCode: integer (nullable = true)
 |-- lastSeen: string (nullable = true)



In [0]:
cars.select('name','seller','model','brand').show(truncate=False)

+-----------------------------------------------------------------+-------+--------+-------------+
|name                                                             |seller |model   |brand        |
+-----------------------------------------------------------------+-------+--------+-------------+
|Zu_verkaufen                                                     |private|3er     |bmw          |
|Volvo_XC90_2.4D_Summum                                           |private|xc_reihe|volvo        |
|Volkswagen_Touran                                                |private|touran  |volkswagen   |
|Seat_Ibiza_1.4_16V_Reference                                     |private|ibiza   |seat         |
|Volvo_XC90_D5_Aut._RDesign_R_Design_AWD_GSHD_Sitzhzg.            |private|xc_reihe|volvo        |
|VW_Passat_35i_Facelift_Limousine_TÜV_Neu                         |private|passat  |volkswagen   |
|Opel_astra                                                       |private|astra   |opel         |
|Mercedes_

In [0]:
cars_fixed = cars.na.fill({'model':'unknown','vehicleType':'default'})

In [0]:
cars_fixed.select('name','model','brand','vehicleType').show(truncate=False)

+-----------------------------------------------------------------+--------+-------------+-------------+
|name                                                             |model   |brand        |vehicleType  |
+-----------------------------------------------------------------+--------+-------------+-------------+
|Zu_verkaufen                                                     |3er     |bmw          |limousine    |
|Volvo_XC90_2.4D_Summum                                           |xc_reihe|volvo        |suv          |
|Volkswagen_Touran                                                |touran  |volkswagen   |bus          |
|Seat_Ibiza_1.4_16V_Reference                                     |ibiza   |seat         |small car    |
|Volvo_XC90_D5_Aut._RDesign_R_Design_AWD_GSHD_Sitzhzg.            |xc_reihe|volvo        |suv          |
|VW_Passat_35i_Facelift_Limousine_TÜV_Neu                         |passat  |volkswagen   |limousine    |
|Opel_astra                                            

In [0]:
cars.show(truncate=False)

+----------------+-----------------------------------------------------------------+-------+---------+-----+-------+-------------+------------------+---------+-------+--------+---------+-------------------+--------+-------------+-----------------+----------------+----------+----------------+
|dateCrawled     |name                                                             |seller |offerType|price|abtest |vehicleType  |yearOfRegistration|gearbox  |powerPS|model   |kilometer|monthOfRegistration|fuelType|brand        |notRepairedDamage|dateCreated     |postalCode|lastSeen        |
+----------------+-----------------------------------------------------------------+-------+---------+-----+-------+-------------+------------------+---------+-------+--------+---------+-------------------+--------+-------------+-----------------+----------------+----------+----------------+
|30/03/2016 13:51|Zu_verkaufen                                                     |private|offer    |4450 |test   |limou

In [0]:
cars_fixed.withColumn('ivm', expr(" seller || '-' || model || '-' || brand")).show()

+----------------+--------------------+-------+---------+-----+-------+-------------+------------------+---------+-------+--------+---------+-------------------+--------+-------------+-----------------+----------------+----------+----------------+--------------------+
|     dateCrawled|                name| seller|offerType|price| abtest|  vehicleType|yearOfRegistration|  gearbox|powerPS|   model|kilometer|monthOfRegistration|fuelType|        brand|notRepairedDamage|     dateCreated|postalCode|        lastSeen|                 ivm|
+----------------+--------------------+-------+---------+-----+-------+-------------+------------------+---------+-------+--------+---------+-------------------+--------+-------------+-----------------+----------------+----------+----------------+--------------------+
|30/03/2016 13:51|        Zu_verkaufen|private|    offer| 4450|   test|    limousine|              2003|   manual|    150|     3er|   150000|                  3|  diesel|          bmw|         

In [0]:
cars_df = cars_fixed.withColumn('ivm', expr(" brand || '-' || model || '-' || vehicleType")).select('name','model','brand',col('dateCrawled').alias('date_created'),'ivm','price',col('vehicleType').alias('type'),col('yearOfRegistration').alias('yr_since_reg'),'postalCode','lastSeen')

In [0]:
cars_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- model: string (nullable = false)
 |-- brand: string (nullable = true)
 |-- date_created: string (nullable = true)
 |-- ivm: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- type: string (nullable = false)
 |-- yr_since_reg: integer (nullable = true)
 |-- postalCode: integer (nullable = true)
 |-- lastSeen: string (nullable = true)



In [0]:
cars_df.show(truncate=True)

+--------------------+--------+-------------+----------------+--------------------+-----+-------------+------------+----------+----------------+
|                name|   model|        brand|    date_created|                 ivm|price|         type|yr_since_reg|postalCode|        lastSeen|
+--------------------+--------+-------------+----------------+--------------------+-----+-------------+------------+----------+----------------+
|        Zu_verkaufen|     3er|          bmw|30/03/2016 13:51|   bmw-3er-limousine| 4450|    limousine|        2003|     20257|07/04/2016 04:44|
|Volvo_XC90_2.4D_S...|xc_reihe|        volvo|07/03/2016 09:54|  volvo-xc_reihe-suv|13299|          suv|        2005|     88045|26/03/2016 13:17|
|   Volkswagen_Touran|  touran|   volkswagen|01/04/2016 00:57|volkswagen-touran...| 3200|          bus|        2003|     27449|01/04/2016 08:40|
|Seat_Ibiza_1.4_16...|   ibiza|         seat|19/03/2016 17:50|seat-ibiza-small car| 4500|    small car|        2006|     34537|07/

In [0]:
cars_df.createOrReplaceTempView('cars_s')

In [0]:
spark.sql('select current_timestamp()').show(10,False)

+-----------------------+
|current_timestamp()    |
+-----------------------+
|2022-07-09 08:51:39.902|
+-----------------------+



In [0]:
spark.sql('select to_unix_timestamp(current_timestamp())').show(10,False)

+-----------------------------------------------------------+
|to_unix_timestamp(current_timestamp(), yyyy-MM-dd HH:mm:ss)|
+-----------------------------------------------------------+
|1657356993                                                 |
+-----------------------------------------------------------+



In [0]:
spark.sql("select ivm, case when date_created is NULL or date_created = ''  then 0 else to_unix_timestamp(date_created,'dd/MM/yyyy HH:mm') end as date_created_uts from cars_s").show(10,False)

+---------------------------+----------------+
|ivm                        |date_created_uts|
+---------------------------+----------------+
|bmw-3er-limousine          |1459345860      |
|volvo-xc_reihe-suv         |1457344440      |
|volkswagen-touran-bus      |1459472220      |
|seat-ibiza-small car       |1458409800      |
|volvo-xc_reihe-suv         |1458139860      |
|volkswagen-passat-limousine|1459601460      |
|opel-astra-station wagon   |1459681560      |
|mercedes_benz-clk-coupe    |1458834720      |
|opel-vectra-station wagon  |1459176000      |
|skoda-octavia-limousine    |1459856280      |
+---------------------------+----------------+
only showing top 10 rows



In [0]:
spark.sql("select ivm, case when date_created is NULL or date_created = ''  then 0 else to_unix_timestamp(date_created,'dd/MM/yyyy HH:mm') end as date_created_uts from cars_s").printSchema()

root
 |-- ivm: string (nullable = true)
 |-- date_created_uts: long (nullable = true)



In [0]:
spark.sql("select ivm, from_unixtime(to_unix_timestamp(date_created,'dd/MM/yyyy HH:mm'), 'yyyy-MM-dd HH:mm:ss.SSSSSS') as date_created from cars_s").show(30,False)

+--------------------------------+--------------------------+
|ivm                             |date_created              |
+--------------------------------+--------------------------+
|bmw-3er-limousine               |2016-03-30 13:51:00.000000|
|volvo-xc_reihe-suv              |2016-03-07 09:54:00.000000|
|volkswagen-touran-bus           |2016-04-01 00:57:00.000000|
|seat-ibiza-small car            |2016-03-19 17:50:00.000000|
|volvo-xc_reihe-suv              |2016-03-16 14:51:00.000000|
|volkswagen-passat-limousine     |2016-04-02 12:51:00.000000|
|opel-astra-station wagon        |2016-04-03 11:06:00.000000|
|mercedes_benz-clk-coupe         |2016-03-24 15:52:00.000000|
|opel-vectra-station wagon       |2016-03-28 14:40:00.000000|
|skoda-octavia-limousine         |2016-04-05 11:38:00.000000|
|mercedes_benz-a_klasse-limousine|2016-03-21 09:52:00.000000|
|opel-astra-coupe                |2016-03-25 12:40:00.000000|
|toyota-yaris-small car          |2016-03-13 07:52:00.000000|
|bmw-3er

In [0]:
spark.sql("select ivm, from_unixtime(to_unix_timestamp(date_created,'dd/MM/yyyy HH:mm'), 'MM-dd-yyyy HH:mm:ss.SSSSSS') as date_created from cars_s").printSchema()

root
 |-- ivm: string (nullable = true)
 |-- date_created: string (nullable = true)



In [0]:
spark.sql("select ivm, to_timestamp(from_unixtime(to_unix_timestamp(date_created,'dd/MM/yyyy HH:mm'), 'MM-dd-yyyy HH:mm:ss.SSSSSS'),'MM-dd-yyyy HH:mm:ss.SSSSSS') as date_created from cars_s").printSchema()

root
 |-- ivm: string (nullable = true)
 |-- date_created: timestamp (nullable = true)



In [0]:
spark.sql("select ivm, to_timestamp(date_created,'dd/MM/yyyy HH:mm') as date_created from cars_s").show(20,False)

+--------------------------------+-------------------+
|ivm                             |date_created       |
+--------------------------------+-------------------+
|bmw-3er-limousine               |2016-03-30 13:51:00|
|volvo-xc_reihe-suv              |2016-03-07 09:54:00|
|volkswagen-touran-bus           |2016-04-01 00:57:00|
|seat-ibiza-small car            |2016-03-19 17:50:00|
|volvo-xc_reihe-suv              |2016-03-16 14:51:00|
|volkswagen-passat-limousine     |2016-04-02 12:51:00|
|opel-astra-station wagon        |2016-04-03 11:06:00|
|mercedes_benz-clk-coupe         |2016-03-24 15:52:00|
|opel-vectra-station wagon       |2016-03-28 14:40:00|
|skoda-octavia-limousine         |2016-04-05 11:38:00|
|mercedes_benz-a_klasse-limousine|2016-03-21 09:52:00|
|opel-astra-coupe                |2016-03-25 12:40:00|
|toyota-yaris-small car          |2016-03-13 07:52:00|
|bmw-3er-default                 |2016-03-21 18:56:00|
|opel-meriva-limousine           |2016-03-08 20:52:00|
|volvo-unk

In [0]:
spark.sql("select ivm, date_format(to_timestamp(date_created,'dd/MM/yyyy HH:mm'),'yyyyMMdd-HHmmss') as date_created from cars_s").show(20,False)

+--------------------------------+---------------+
|ivm                             |date_created   |
+--------------------------------+---------------+
|bmw-3er-limousine               |20160330-135100|
|volvo-xc_reihe-suv              |20160307-095400|
|volkswagen-touran-bus           |20160401-005700|
|seat-ibiza-small car            |20160319-175000|
|volvo-xc_reihe-suv              |20160316-145100|
|volkswagen-passat-limousine     |20160402-125100|
|opel-astra-station wagon        |20160403-110600|
|mercedes_benz-clk-coupe         |20160324-155200|
|opel-vectra-station wagon       |20160328-144000|
|skoda-octavia-limousine         |20160405-113800|
|mercedes_benz-a_klasse-limousine|20160321-095200|
|opel-astra-coupe                |20160325-124000|
|toyota-yaris-small car          |20160313-075200|
|bmw-3er-default                 |20160321-185600|
|opel-meriva-limousine           |20160308-205200|
|volvo-unknown-station wagon     |20160402-134400|
|nissan-others-suv             

In [0]:
spark.sql("select ivm, cast(date_format(to_timestamp(date_created,'dd/MM/yyyy HH:mm'),'yyyyMMddHHmmss') as integer) as date_created from cars_s").printSchema()

root
 |-- ivm: string (nullable = true)
 |-- date_created: integer (nullable = true)



In [0]:
spark.sql("select ivm, datediff(to_timestamp(lastSeen,'dd/MM/yyyy HH:mm'),to_timestamp(date_created,'dd/MM/yyyy HH:mm')) as days_lasted , \
date_format(date_add(to_timestamp(date_created,'dd/MM/yyyy HH:mm'), 1),'yyyy-MM-dd HH:mm:ss') as next_day from cars_s").show(20,False)

+--------------------------------+-----------+-------------------+
|ivm                             |days_lasted|next_day           |
+--------------------------------+-----------+-------------------+
|bmw-3er-limousine               |8          |2016-03-31 00:00:00|
|volvo-xc_reihe-suv              |19         |2016-03-08 00:00:00|
|volkswagen-touran-bus           |0          |2016-04-02 00:00:00|
|seat-ibiza-small car            |19         |2016-03-20 00:00:00|
|volvo-xc_reihe-suv              |16         |2016-03-17 00:00:00|
|volkswagen-passat-limousine     |0          |2016-04-03 00:00:00|
|opel-astra-station wagon        |3          |2016-04-04 00:00:00|
|mercedes_benz-clk-coupe         |3          |2016-03-25 00:00:00|
|opel-vectra-station wagon       |9          |2016-03-29 00:00:00|
|skoda-octavia-limousine         |0          |2016-04-06 00:00:00|
|mercedes_benz-a_klasse-limousine|1          |2016-03-22 00:00:00|
|opel-astra-coupe                |3          |2016-03-26 00:00

In [0]:
spark.sql("SELECT date_part('dow', DATE'2019-08-12')").show(1,False)

+---------------------------------+
|date_part(dow, DATE '2019-08-12')|
+---------------------------------+
|2                                |
+---------------------------------+



In [0]:
spark.sql("SELECT date_part('dow',to_timestamp(date_created,'dd/MM/yyyy HH:mm') ) as day_of_week from cars_s").show(30,False)

+-----------+
|day_of_week|
+-----------+
|4          |
|2          |
|6          |
|7          |
|4          |
|7          |
|1          |
|5          |
|2          |
|3          |
|2          |
|6          |
|1          |
|2          |
|3          |
|7          |
|7          |
|3          |
|3          |
|6          |
|6          |
|1          |
|7          |
|1          |
|7          |
|1          |
|7          |
|7          |
|3          |
|7          |
+-----------+
only showing top 30 rows



In [0]:
#date APIS
cars_df.select('ivm', F.dayofweek(F.to_timestamp(col('date_created'),'dd/MM/yyyy HH:mm')).alias('day_of_week')).show(31,False)

+--------------------------------+-----------+
|ivm                             |day_of_week|
+--------------------------------+-----------+
|bmw-3er-limousine               |4          |
|volvo-xc_reihe-suv              |2          |
|volkswagen-touran-bus           |6          |
|seat-ibiza-small car            |7          |
|volvo-xc_reihe-suv              |4          |
|volkswagen-passat-limousine     |7          |
|opel-astra-station wagon        |1          |
|mercedes_benz-clk-coupe         |5          |
|opel-vectra-station wagon       |2          |
|skoda-octavia-limousine         |3          |
|mercedes_benz-a_klasse-limousine|2          |
|opel-astra-coupe                |6          |
|toyota-yaris-small car          |1          |
|bmw-3er-default                 |2          |
|opel-meriva-limousine           |3          |
|volvo-unknown-station wagon     |7          |
|nissan-others-suv               |7          |
|others-unknown-others           |3          |
|volkswagen-g

In [0]:
cars_df.select('ivm', F.datediff(F.to_timestamp(col('lastSeen'),'dd/MM/yyyy HH:mm'),F.to_timestamp(col('date_created'),'dd/MM/yyyy HH:mm')).alias('days_lasted')).show(31,False)

+--------------------------------+-----------+
|ivm                             |days_lasted|
+--------------------------------+-----------+
|bmw-3er-limousine               |8          |
|volvo-xc_reihe-suv              |19         |
|volkswagen-touran-bus           |0          |
|seat-ibiza-small car            |19         |
|volvo-xc_reihe-suv              |16         |
|volkswagen-passat-limousine     |0          |
|opel-astra-station wagon        |3          |
|mercedes_benz-clk-coupe         |3          |
|opel-vectra-station wagon       |9          |
|skoda-octavia-limousine         |0          |
|mercedes_benz-a_klasse-limousine|1          |
|opel-astra-coupe                |3          |
|toyota-yaris-small car          |0          |
|bmw-3er-default                 |2          |
|opel-meriva-limousine           |28         |
|volvo-unknown-station wagon     |4          |
|nissan-others-suv               |12         |
|others-unknown-others           |15         |
|volkswagen-g

In [0]:
cars_df.withColumn('formated_date_create', F.date_format(F.to_timestamp(col('date_created'),'dd/MM/yyyy HH:mm'),'yyy-MM-dd HH:mm:ss.SSSS'))\
.select('ivm',F.trunc(col('formated_date_create'),'year').substr(1,4).alias('year_created'),'formated_date_create').show(30,False)

+--------------------------------+------------+------------------------+
|ivm                             |year_created|formated_date_create    |
+--------------------------------+------------+------------------------+
|bmw-3er-limousine               |2016        |2016-03-30 13:51:00.0000|
|volvo-xc_reihe-suv              |2016        |2016-03-07 09:54:00.0000|
|volkswagen-touran-bus           |2016        |2016-04-01 00:57:00.0000|
|seat-ibiza-small car            |2016        |2016-03-19 17:50:00.0000|
|volvo-xc_reihe-suv              |2016        |2016-03-16 14:51:00.0000|
|volkswagen-passat-limousine     |2016        |2016-04-02 12:51:00.0000|
|opel-astra-station wagon        |2016        |2016-04-03 11:06:00.0000|
|mercedes_benz-clk-coupe         |2016        |2016-03-24 15:52:00.0000|
|opel-vectra-station wagon       |2016        |2016-03-28 14:40:00.0000|
|skoda-octavia-limousine         |2016        |2016-04-05 11:38:00.0000|
|mercedes_benz-a_klasse-limousine|2016        |2016